In [72]:
#importing libraries
import pandas as pd
import numpy as np
import mysql.connector
from datetime import datetime, timedelta

In [73]:
from faker import Faker
fake = Faker('en_IN')

In [74]:
#connection to sql database
import sqlalchemy, pymysql
from sqlalchemy.dialects import mysql
import pandas as pd
db = sqlalchemy.create_engine(
     sqlalchemy.engine.url.URL(
          drivername='mysql+pymysql',
          username='root',
          password="Yamini@kvt12",
          host='35.196.234.121',
          database="ufh-db"
     )
)

C:\Users\pothula.yamini\AppData\Local\Temp\ipykernel_392\2382207158.py:5: SADeprecationWarning: Calling URL() directly is deprecated and will be disabled in a future release.  The public constructor for URL is now the URL.create() method.
  sqlalchemy.engine.url.URL(


In [123]:
#fetching max customer_id from customer_table in sql instance
qry1='select max(customerid) as maxcustid from CUSTOMER_MASTER;'
maxcustid= pd.read_sql(qry1,con=db)
maxcustid=maxcustid.iat[0,0]

In [124]:
#fetching max product_id from product_table in sql instance
qry2='select max(Productid) from PRODUCT_MASTER;'
maxprodid= pd.read_sql(qry2,con=db)
maxprodid=maxprodid.iat[0,0]

In [125]:
#fetching max order_id from order_details table in sql instance
qry3='select max(orderid) from ORDER_DETAILS;'
orderid= pd.read_sql(qry3,con=db)
orderid=orderid.iat[0,0]
print(orderid)

30000


# incremental order details

In [126]:
#creating order_recieved dataframe of 5000 orders
order_detail_fields = ['orderid', 'customerid', 'order_status_update_timestamp', 'order_status']
orders_Received = pd.DataFrame(columns=order_detail_fields, index=range(1, 15001))

x=orderid+1
j=1
for i in range(1, 5001):
    f=fake.random_int(min=1, max=maxcustid)
    for k in range (j,j+3):
        orders_Received['orderid'][k] = x
        orders_Received['customerid'][k]=f

    orders_Received['order_status_update_timestamp'][j] = fake.date_time_this_year()
    orders_Received['order_status'][j] = 'Received'
    orders_Received['order_status_update_timestamp'][j+1] = orders_Received['order_status_update_timestamp'][j] + timedelta(seconds=fake.random_int(min=1, max=86400))
    orders_Received['order_status'][j+1] = 'In_progress'
    orders_Received['order_status_update_timestamp'][j+2] = orders_Received['order_status_update_timestamp'][j+1] + timedelta(seconds=fake.random_int(min=1, max=172800))
    orders_Received['order_status'][j+2] = 'Delivered'
    j+=3
    x+=1

In [127]:
orders_Received.head(-10)

,orderid,customerid,order_status_update_timestamp,order_status
1,30001,914,2022-03-19 22:22:44,Received
2,30001,914,2022-03-20 14:47:03,In_progress
3,30001,914,2022-03-22 13:55:17,Delivered
4,30002,444,2022-10-10 14:08:21,Received
5,30002,444,2022-10-10 20:36:53,In_progress
...,...,...,...,...
14986,34996,830,2022-02-17 15:34:24,Received
14987,34996,830,2022-02-17 18:35:05,In_progress
14988,34996,830,2022-02-18 21:50:16,Delivered
14989,34997,242,2022-04-28 01:54:10,Received


## loading

In [128]:
#appending dataframe into order_details table
orders_Received.to_sql('ORDER_DETAILS', con=db, if_exists='append',index=False)

15000

# incremental order items

In [129]:
order_item_fields = ['orderid','productid','quantity']

In [130]:
order_items = pd.DataFrame(columns=order_item_fields, index = range(1,5001))

In [131]:
for i in range(1,5001):
    order_items['orderid'][i]=fake.random_element(orders_Received['orderid'])
    order_items['productid'][i]=fake.random_int(min=1, max=maxprodid)
    order_items['quantity'][i]=fake.random_int(min=1, max=20)

In [132]:
order_items = order_items.drop_duplicates(subset=['orderid', 'productid'],keep='first')

In [133]:
order_items.head(-10)

,orderid,productid,quantity
1,33695,179,5
2,33070,118,12
3,34979,36,1
4,31580,51,12
5,32710,82,18
...,...,...,...
4986,33286,112,7
4987,34901,158,8
4988,31900,41,11
4989,32880,122,6


## loading

In [134]:
order_items.to_sql('ORDER_ITEMS', con=db, if_exists='append',index=False)

4992